In [60]:
import sklearn

import numpy as np
import sys
sys.path.append(r'/home/jupyter/reusable_code')
import google_api_functions as gaf
from google.cloud import bigquery # To run BQ statements
creds=gaf.Authenticate_Google(r'/home/jupyter/reusable_code/') # Return logged-in credentials

from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import pandas as pd
from sklearn import metrics

import plotly.express as px

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:

bq = bigquery.Client(project='itv-bde-analytics-dev',credentials=creds)
query='''select * from 
`itv-bde-analytics-prd.britbox_sandbox.churnbyViewsegment_UNPIVOT`'''
df = bq.query(query).to_dataframe()
df.head()

In [17]:
df=df[df['ttl_progs']>0].fillna(0)

df.head()

In [18]:
df.columns

In [19]:

# Create a function to map a colour to cluster number
def name_cluster(x):
    if x ==0:
        return 'Medium X-sell, High Engagement (Key Titles)'
    elif x ==1:
        return 'High X-sell, Low Engagement'
    elif x ==2:
        return 'High X-sell, Medium Engagement'
    elif x ==3:
        return 'High X-sell, Very Low Engagement'
    elif x ==4:
        return 'Low X-sell, High Engagement (Love Island)'
    elif x ==5:
        return 'High X-sell, High Engagement'
    elif x ==99:
        return 'Not in a segment'
    else:
        return 'Unknown'
#df['clusterName']= df['clusterNum'].apply(get_colour)


In [20]:
df[df['ttl_progs']<12].ttl_progs.hist()
plt.title('Histogram of NumProgs')
plt.xlabel('NumProgs')
plt.ylabel('Frequency')
plt.show()

In [21]:
df.cancel_flag.hist()

In [22]:
xvars=[i for i in df.columns if i[:10]=='pc_streams']
xvars

### So the regression doesn't work. Is the mix different by repertoire?

In [24]:
varlist=[xvars][0]+['cancel_flag','ttl_progs']
varlist


In [25]:
mixdf=df[varlist][df['ttl_progs']<11].groupby('ttl_progs').mean()
mixdf

In [28]:

fig = px.bar(mixdf, x=mixdf.index, y=xvars, title="Wide-Form Input")
fig.show()


In [29]:
mixdf

## Re run splitting by bill month

In [49]:
varlist=[xvars][0]+['cancel_flag','ttl_progs','bill_month']
mixdf2=df[varlist][df['ttl_progs']<11].groupby(['bill_month','ttl_progs']).mean()
mixdf2

In [63]:

for i in range(1,11):
    fig = px.bar(mixdf2.loc[i], x=mixdf.index, y=xvars, title="Distribution of Repertoire vs Viewing Segments")
 
    fig.add_trace(go.Scatter(x=mixdf.index, y=mixdf['cancel_flag'], name="Cancellation Rate",text='Labels'))

    fig.update_layout(title_text='Bill Month {}'.format(i), title_x=0.5)
    fig.show()

In [48]:
mixdf2.unstack()